# Developing Machine Learning Model for Early Detection of Heart Failure

### Setting up my environment for machine learning with some common libraries.

In [ ]:
# General Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Scikit-learn Libraries
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, log_loss, precision_recall_curve
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.datasets import make_classification
from sklearn.svm import SVC
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.multioutput import ClassifierChain
from sklearn.feature_selection import RFE, mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import label_binarize
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.cluster import DBSCAN
# SciPy Libraries
from scipy.stats import fisher_exact
from scipy.stats.mstats import winsorize

# Statsmodels Library
from statsmodels.stats.contingency_tables import mcnemar
from bs4 import BeautifulSoup
### Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
%pip install gdown

  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached beautifulsoup4-4.13.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
Using cached beautifulsoup4-4.13.3-py3-none-any.whl (186 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


## Data Preprocessing

### Load the Data Set

In [4]:
data = pd.read_csv("HF Data.csv", encoding='ISO-8859-1')
data.head()

,StudyID,UHID,SN,Acute Coronary Syndrome (ACS),Irregular Heartbeat (Arrhythmia),Bronchial Asthma (BA),Coronary Artery Disease (CAD),Chest X-ray (CXR),Dyslipidemia : Abnormal Cholesterol Levels (DL),Diabetes Mellitus (DM),...,Sodium (mmol/L) (Na),Random Blood Sugar (mg/dL) (RBS),Systolic Blood Pressure (mmHg) (SBP),Total Cholesterol (mg/dL) (TC),Triglycerides (mg/dL) (TG),Troponin I (TropI),Weight (kg),Obisity,Chest_pain,Heart Failure (HF)
0,MPI001,205179,1,ST-Elevation Myocardial Infarction (STEMI),No Arrhythmia,No,Normal,Normal,No,No,...,140,4.9,180,100,95,0.001,60,Obese,Absent,No HF
1,MPI002,311074,2,ST-Elevation Myocardial Infarction (STEMI),No Arrhythmia,No,Normal,Normal,No,No,...,137,6.5,140,110,85,0.005,64,No Obese,Absent,No HF
2,MPI003,530942,3,ST-Elevation Myocardial Infarction (STEMI),No Arrhythmia,No,Normal,Normal,No,No,...,139,4.9,140,130,100,6.500,85,No Obese,Absent,No HF
3,MPI004,3414,4,ST-Elevation Myocardial Infarction (STEMI),No Arrhythmia,No,Normal,Normal,No,Yes,...,141,5.4,130,135,120,0.001,66,No Obese,Absent,No HF
4,MPI005,777747,5,ST-Elevation Myocardial Infarction (STEMI),No Arrhythmia,No,Normal,Normal,Yes,No,...,141,6.0,130,220,190,0.003,70,Obese,Absent,No HF


### Generate a profiling report

In [8]:
# Install the ydata-profiling package
%pip install ydata-profiling

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Generate a profiling report
from ydata_profiling import ProfileReport  # Import ProfileReport

profile = ProfileReport(data, title="Dataset Profiling Report", explorative=True)
profile

ModuleNotFoundError: No module named 'ipywidgets'

### Drop unnecessary columns

In [ ]:
# Drop unnecessary columns
data =data.drop(columns=['StudyID'])

In [ ]:
data.columns

Index(['Age', 'Sex', 'BMI', 'NYHA', 'HR', 'HTN', 'DM', 'Smoker', 'DL', 'BA',
       'RBS', 'HbA1C', 'Creatinine', 'Na', 'K', 'Cl', 'Hb', 'TropI', 'CXR',
       'ECG', 'LVIDd', 'FS', 'LVIDs', 'LVEF', 'RWMA', 'LAV', 'MI', 'ACS',
       'Wall', 'Thrombolysis', 'ICT', 'IRT', 'MR', 'EA', 'DT', 'MPI', 'RR',
       'Chest_pain', 'TC', 'LDLc', 'HDLc', 'TG', 'BNP', 'HF'],
      dtype='object')

## Summery of Statistical Numberical Colums before remove outliers

In [ ]:
data.describe(include=[np.number]).T

,count,mean,std,min,25%,50%,75%,max
Age,500.0,55.530000,12.696121,18.00,47.000,56.000,65.000,95.00
BMI,500.0,25.378720,4.048717,12.26,22.815,24.800,27.550,48.89
HR,500.0,80.706000,14.923706,30.00,72.000,78.000,88.000,153.00
RBS,500.0,8.227780,3.536930,4.00,5.800,7.000,9.650,28.70
HbA1C,500.0,6.274600,1.769757,0.00,5.300,5.600,7.125,13.40
Creatinine,500.0,1.332320,1.278408,0.50,0.960,1.105,1.300,15.08
Na,500.0,138.470000,3.909433,123.00,136.000,138.000,141.000,149.00
K,500.0,3.915160,0.353988,3.00,3.700,3.900,4.100,6.00
Cl,500.0,101.608000,5.190010,90.00,99.000,103.000,105.000,110.00
Hb,500.0,12.463200,1.469055,7.80,11.300,12.300,13.500,16.70


### Cheak data Shape

In [ ]:
data.shape

(500, 44)

## Cheak missing values

In [ ]:
# Check for missing values
missing_values = data.isnull().sum()

# Display columns with missing values
print("Missing Values in Each Column:")
print(missing_values[missing_values > 0])

# Display the total number of missing values
total_missing = data.isnull().sum().sum()
print(f"\nTotal Missing Values in the Dataset: {total_missing}")

Missing Values in Each Column:
Series([], dtype: int64)

Total Missing Values in the Dataset: 0


## Cheaking duplicate columns

In [ ]:
# Check for duplicate columns
duplicate_columns = data.columns[data.columns.duplicated()].unique()

# Display the duplicate columns (if any)
if len(duplicate_columns) > 0:
    print(f"Duplicate columns found: {duplicate_columns}")
else:
    print("No duplicate columns found.")

No duplicate columns found.


## Check for duplicate rows

In [ ]:
# Check for duplicate rows
duplicate_rows = data[data.duplicated()]

# Display the duplicate rows (if any)
if not duplicate_rows.empty:
    print("Duplicate rows found:")
    print(duplicate_rows)
else:
    print("No duplicate rows found.")


No duplicate rows found.


### Outliners decetion and Removal

Which Method is Best for Heart Disease Prediction?

If the dataset is small and structured: Use IQR or Z-Score.

If the dataset has high-dimensional features: Use Isolation Forest or LOF.

If there are non-linear patterns: Use DBSCAN.

 A common approach in medical datasets is to log transform certain features (e.g., glucose levels, blood pressure) that have highly skewed distributions.

#### Plot Oulters

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set up the plotting grid
num_cols = data.select_dtypes(include=[np.number]).columns
fig, axes = plt.subplots(nrows=len(num_cols), ncols=2, figsize=(10, len(num_cols)*5))

# Loop through numeric columns and plot histograms and scatter plots
for i, col in enumerate(num_cols):
    # Histogram
    sns.histplot(data[col], kde=True, ax=axes[i, 0])
    axes[i, 0].set_title(f'Histogram of {col}')

    # Detect outliers using IQR method
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]

    # Scatter plot with outliers in a different color
    sns.scatterplot(x=data.index, y=data[col], ax=axes[i, 1], color=['skyblue'])  # Normal points
    sns.scatterplot(x=outliers.index, y=outliers[col], ax=axes[i, 1], color='red')  # Outliers in red
    axes[i, 1].set_title(f'Scatter Plot of {col} with Outliers')

plt.tight_layout()
plt.show()


## Classification based on Skewness and Normality Test:
Normal Distribution: If the skewness is near 0 and the p-value from the normality test is greater than 0.05, the feature is classified as normally distributed.

Right-skewed: Features with a positive skewness greater than 0.5 are considered right-skewed.

Left-skewed: Features with a negative skewness less than -0.5 are considered left-skewed.

Non-normal: Features that fail the normality test (p_value < 0.05)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, normaltest

# Assuming `data` is your dataset (e.g., a pandas DataFrame)

# Initialize empty lists to store the classification results
normal_dist = []
right_skewed = []
left_skewed = []
non_normal_dist = []  # List to store non-normal features

# Iterate over each numeric column in the dataset
for column in data.select_dtypes(include=[np.number]).columns:
    # Calculate skewness
    feature_skewness = skew(data[column].dropna())  # Drop NaN values before calculation

    # Check for normality using D'Agostino's K-squared test (normality test)
    _, p_value = normaltest(data[column].dropna())  # p_value < 0.05 means the data is not normal

    if abs(feature_skewness) < 0.5 and p_value > 0.05:
        # If skewness is close to 0 and data passes the normality test
        normal_dist.append(column)
    elif feature_skewness > 0.5:
        # If skewness is positive (right skewed)
        right_skewed.append(column)
    elif feature_skewness < -0.5:
        # If skewness is negative (left skewed)
        left_skewed.append(column)

    # Classify as non-normal if the p-value is < 0.05 (fails normality test)
    if p_value < 0.05:
        non_normal_dist.append(column)

# Print the results
print("Normal Distributed Features:")
print(normal_dist)

print("\nRight Skewed Features:")
print(right_skewed)

print("\nLeft Skewed Features:")
print(left_skewed)

print("\nNon-Normal Distributed Features (failed normality test):")
print(non_normal_dist)


Normal Distributed Features:
['Age', 'ICT']

Right Skewed Features:
['BMI', 'HR', 'RBS', 'HbA1C', 'Creatinine', 'K', 'TropI', 'LVIDs', 'EA', 'DT', 'MPI', 'RR', 'TC', 'LDLc', 'HDLc', 'TG', 'BNP']

Left Skewed Features:
['Cl']

Non-Normal Distributed Features (failed normality test):
['BMI', 'HR', 'RBS', 'HbA1C', 'Creatinine', 'Na', 'K', 'Cl', 'Hb', 'TropI', 'LVIDd', 'FS', 'LVIDs', 'LVEF', 'LAV', 'IRT', 'EA', 'DT', 'MPI', 'RR', 'TC', 'LDLc', 'HDLc', 'TG', 'BNP']


 ### Transformations and Outliers:

Log Transformation: Reduces the impact of high right-skewed outliers.

Square Root Transformation: Helps with moderate skewness.

Box-Cox Transformation: Transforms data into a more normal distribution, which is helpful for handling both positive and negative skewness.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

# Assuming `data` is your DataFrame and you already have skewed features identified

# Create a copy of the original data to keep track of changes
data_transformed = data.copy()

# Function to detect outliers using IQR method

def detect_outliers_iqr(df):
    outliers = {}
    # Select only numerical columns for outlier detection
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for column in numeric_cols:  # Iterate through numerical columns
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[column] = ((df[column] < lower_bound) | (df[column] > upper_bound)).sum()
    return outliers

# Print before transformation
print("Before Transformation:")
print(data.describe())

# Check outliers before transformation
outliers_before = detect_outliers_iqr(data)
print("\nOutliers Before Transformation:")
print(outliers_before)

# 1. Apply Log Transformation to Right-Skewed Features
right_skewed = ['BMI', 'HR', 'RBS', 'Creatinine', 'K', 'LVIDs', 'MR', 'MPI', 'RR', 'TC', 'LDLc', 'HDLc', 'TG']

for column in right_skewed:
    data[column] = pd.to_numeric(data[column], errors='coerce')  # Ensure numeric values
    if data[column].notna().all() and (data[column] > 0).all():
        data_transformed[column] = np.log(data[column] + 1)

# 2. Apply Square Root or Cube Root Transformation to Moderately Skewed Features (Optional)
moderately_skewed = ['HbA1C', 'Cl', 'TropI', 'BNP']  # Example for left-skewed features

for column in moderately_skewed:
    if (data[column] > 0).all():  # Ensure no negative or zero values
        data_transformed[column] = np.sqrt(data[column] + 1)  # Square root transformation for moderate skewness

# 3. Apply Box-Cox Transformation to Non-Normal Distributed Features
# Box-Cox works best when the data is strictly positive
non_normal_dist = [ 'Na','K','Hb','LVIDd','FS','LVEF','LAV','IRT','EA','DT','RR']

for column in non_normal_dist:
    if (data[column] > 0).all():  # Apply only if data is strictly positive
        data_transformed[column], _ = stats.boxcox(data[column] + 1)  # Add 1 to avoid log(0)

# Print after transformation
print("\nAfter Transformation:")
print(data_transformed.describe())

# Check outliers after transformation
outliers_after = detect_outliers_iqr(data_transformed)
print("\nOutliers After Transformation:")
print(outliers_after)


Before Transformation:
              Age         BMI          HR        RBS       HbA1C  Creatinine  \
count  500.000000  500.000000  500.000000  500.00000  500.000000  500.000000   
mean    55.530000   25.378720   80.706000    8.22778    6.274600    1.332320   
std     12.696121    4.048717   14.923706    3.53693    1.769757    1.278408   
min     18.000000   12.260000   30.000000    4.00000    0.000000    0.500000   
25%     47.000000   22.815000   72.000000    5.80000    5.300000    0.960000   
50%     56.000000   24.800000   78.000000    7.00000    5.600000    1.105000   
75%     65.000000   27.550000   88.000000    9.65000    7.125000    1.300000   
max     95.000000   48.890000  153.000000   28.70000   13.400000   15.080000   

               Na           K         Cl          Hb  ...        IRT  \
count  500.000000  500.000000  500.00000  500.000000  ...  500.00000   
mean   138.470000    3.915160  101.60800   12.463200  ...   98.03600   
std      3.909433    0.353988    5.19001

Distribution of Brfore and After Log Transformation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Loop over each numeric feature to plot
for feature in data.select_dtypes(include=[np.number]).columns:
    # Create a figure with 2 subplots (side by side)
    plt.figure(figsize=(14, 6))

    # Plot Before Transformation
    plt.subplot(1, 2, 1)  # 1 row, 2 columns, this is the first subplot
    sns.histplot(data[feature], kde=True)
    plt.title(f'Distribution of {feature} Before Transformation')

    # Plot After Transformation
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, this is the second subplot
    sns.histplot(data_transformed[feature], kde=True)
    plt.title(f'Distribution of {feature} After Log Transformation')

    # Show both plots
    plt.tight_layout()  # Adjust layout to avoid overlap
    plt.show()


Function to apply capping (winsorization)

In [ ]:
# Function to apply capping (winsorization)
def cap_outliers(df, columns, lower_percentile=0.01, upper_percentile=0.99):
    for column in columns:
        lower_limit = df[column].quantile(lower_percentile)
        upper_limit = df[column].quantile(upper_percentile)
        df[column] = np.where(df[column] < lower_limit, lower_limit, df[column])
        df[column] = np.where(df[column] > upper_limit, upper_limit, df[column])
    return df

# Apply capping to all numerical columns
numeric_cols = data_transformed.select_dtypes(include=[np.number]).columns
data_transformed = cap_outliers(data_transformed, numeric_cols)

# Check outliers after capping
outliers_after_capping = detect_outliers_iqr(data_transformed)
print("\nOutliers After Capping:")
print(outliers_after_capping)



Outliers After Capping:
{'Age': 0, 'BMI': 8, 'HR': 21, 'RBS': 7, 'HbA1C': 32, 'Creatinine': 28, 'Na': 9, 'K': 18, 'Cl': 52, 'Hb': 0, 'TropI': 55, 'LVIDd': 8, 'FS': 0, 'LVIDs': 0, 'LVEF': 0, 'LAV': 0, 'ICT': 0, 'IRT': 0, 'MR': 0, 'EA': 43, 'DT': 6, 'MPI': 9, 'RR': 0, 'TC': 0, 'LDLc': 0, 'HDLc': 28, 'TG': 0, 'BNP': 70}


## Descriptive Statistics

### Summery of Statistical Numberical Colums

In [ ]:
data.describe(include=[np.number]).T

,count,mean,std,min,25%,50%,75%,max
Age,500.0,55.530000,12.696121,18.00,47.000,56.000,65.000,95.00
BMI,500.0,25.378720,4.048717,12.26,22.815,24.800,27.550,48.89
HR,500.0,80.706000,14.923706,30.00,72.000,78.000,88.000,153.00
RBS,500.0,8.227780,3.536930,4.00,5.800,7.000,9.650,28.70
HbA1C,500.0,6.274600,1.769757,0.00,5.300,5.600,7.125,13.40
Creatinine,500.0,1.332320,1.278408,0.50,0.960,1.105,1.300,15.08
Na,500.0,138.470000,3.909433,123.00,136.000,138.000,141.000,149.00
K,500.0,3.915160,0.353988,3.00,3.700,3.900,4.100,6.00
Cl,500.0,101.608000,5.190010,90.00,99.000,103.000,105.000,110.00
Hb,500.0,12.463200,1.469055,7.80,11.300,12.300,13.500,16.70


#### Summery of Statistical catagorical Colums

In [ ]:
data.describe(include=['object', 'category']).T  #No objects to concatenate

,count,unique,top,freq
Sex,500,2,1,358
NYHA,500,4,2,182
HTN,500,2,1,318
DM,500,2,0,269
Smoker,500,2,0,320
DL,500,2,1,282
BA,500,2,0,452
CXR,500,2,0,392
ECG,500,5,0,176
RWMA,500,2,1,345


Plotting for categorical variables

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Get the column names from the X_test DataFrame
feature_names =data.columns.tolist()

# Set up the plotting for categorical variables
cat_cols = data.select_dtypes(include=["object",'category']).columns

# Loop through categorical columns and plot separate bar plots for each feature
for col in cat_cols:
    plt.figure(figsize=(10, 5))  # Create a new figure for each feature

    # Countplot with multiple colors
    sns.countplot(x=data[col], palette=['skyblue','lightcoral','red','olive'])  # You can change the palette to any you prefer

    # Title for the plot
    plt.title(f'Bar Plot of {col}')

    # Calculate percentages
    total = len(data[col])
    ax = plt.gca()  # Get current axes
    for p in ax.patches:
        height = p.get_height()
        percentage = (height / total) * 100
        ax.text(p.get_x() + p.get_width() / 2, height + 1, f'{percentage:.2f}%', ha='center', va='bottom')

    # Rotate x-axis labels to be vertical
    plt.xticks(rotation=90)

    plt.tight_layout()  # Adjust layout for better spacing
    plt.show()  # Show the plot for each feature


### Define the terget Variable

In [ ]:
data['HF'].unique()

array(['0', '1'], dtype=object)

In [ ]:
# Data
HF_counts = data['HF'].value_counts()
HF_percentage = HF_counts / HF_counts.sum() * 100

# Pie Chart with Shadows (Simulated 3D Effect)
plt.figure(figsize=(6, 4))
colors = ['skyblue','lightcoral','red','olive'][:len(HF_counts)]

plt.pie(
    HF_counts,
    labels=HF_counts.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    explode=[0.1] * len(HF_counts),  # Slightly separate each slice
    shadow=True  # Add shadow for depth
)
plt.title('Heart Failure (HF) Status')
plt.axis('equal')  # Ensures the pie chart is circular
plt.show()

In [ ]:
# Bar Chart with Enhancements
fig, ax = plt.subplots(figsize=(5, 4))
colors = sns.color_palette("cool", len(HF_counts))  # Gradient color palette

bars = ax.bar(HF_counts.index, HF_counts.values, color=['deepskyblue','lightcoral','red','olive'], edgecolor='white', linewidth=1.2)

# Annotate bars
for bar, percentage in zip(bars, HF_percentage):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.5,
        f'{int(bar.get_height())} ({percentage:.1f}%)',
        ha='center',
        va='bottom',
        fontsize=10
    )

ax.set_title('Heart Failure (HF)', fontsize=14, weight='bold')
ax.set_ylabel('Count', fontsize=12)
ax.set_xlabel('Heart Failure (HF)', fontsize=12)
plt.xticks(rotation=0, fontsize=10)
sns.despine()  # Remove top and right spines for a cleaner look
plt.show()

Basic Statistics Summary

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming your dataset is stored in a DataFrame called `df`

# 1. Basic Statistics Summary using describe()
print("Basic Statistics of the Dataset:")
print(data.describe(include='all'))  # Describe all columns, including categorical features

# 2. Check for missing values in each column
print("\nMissing Values in the Dataset:")
print(data.isnull().sum())  # Sum of missing values per column

# 3. Data Types of the Columns
print("\nData Types of the Columns:")
print(data.dtypes)


# 5. Check for unique values in each column
print("\nUnique Values per Column:")
print(data.nunique())





Basic Statistics of the Dataset:
               Age  Sex         BMI NYHA          HR  HTN   DM Smoker   DL  \
count   500.000000  500  500.000000  500  500.000000  500  500    500  500   
unique         NaN    2         NaN    4         NaN    2    2      2    2   
top            NaN    1         NaN    2         NaN    1    0      0    1   
freq           NaN  358         NaN  182         NaN  318  269    320  282   
mean     55.530000  NaN   25.378720  NaN   80.706000  NaN  NaN    NaN  NaN   
std      12.696121  NaN    4.048717  NaN   14.923706  NaN  NaN    NaN  NaN   
min      18.000000  NaN   12.260000  NaN   30.000000  NaN  NaN    NaN  NaN   
25%      47.000000  NaN   22.815000  NaN   72.000000  NaN  NaN    NaN  NaN   
50%      56.000000  NaN   24.800000  NaN   78.000000  NaN  NaN    NaN  NaN   
75%      65.000000  NaN   27.550000  NaN   88.000000  NaN  NaN    NaN  NaN   
max      95.000000  NaN   48.890000  NaN  153.000000  NaN  NaN    NaN  NaN   

         BA  ...          DT  

In [ ]:
dataf=data

### Apply Model Without Fearures selections(or With 47 Columns)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Function to perform encoding based on the recommended encoding method
def encode_columns(data):
    # Label Encoder initialization
    le = LabelEncoder()

    # Define the columns and their respective encoding methods
    label_encoding_columns = [
        'Sex', 'NYHA','HTN','DM', 'Smoker', 'DL', 'BA','CXR','RWMA', 'MI',
        'Chest_pain', 'HF'
    ]

    one_hot_encoding_columns = [
        'ECG', 'ACS', 'Wall','MR','Thrombolysis','MR'
    ]

    # Apply Label Encoding to binary categorical variables
    for col in label_encoding_columns:
        data[col] = le.fit_transform(data[col])

    # Apply One-Hot Encoding to nominal categorical variables
    data = pd.get_dummies(data, columns=one_hot_encoding_columns, drop_first=True)

    return data # Changed 'df' to 'data' to return the modified DataFrame

# Encode the dataset
encoded_data = encode_columns(data)

# Display the encoded dataset
print(encoded_data)

      Age  Sex    BMI  NYHA    HR  HTN  DM  Smoker  DL  BA  ...  Wall_9  \
0    50.0    0  28.72     0  74.0    1   1       1   1   0  ...    True   
1    74.0    1  23.88     0  84.0    0   1       0   1   0  ...    True   
2    51.0    1  25.24     0  78.0    0   1       0   1   0  ...   False   
3    68.0    1  27.85     0  72.0    1   1       0   0   0  ...   False   
4    34.0    1  21.61     0  72.0    0   0       1   0   0  ...   False   
..    ...  ...    ...   ...   ...  ...  ..     ...  ..  ..  ...     ...   
495  73.0    0  40.06     2  69.0    1   1       0   1   0  ...   False   
496  35.0    0  36.95     3  78.0    1   0       0   1   0  ...   False   
497  62.0    1  21.22     0  75.0    1   0       1   0   0  ...    True   
498  61.0    1  22.95     1  80.0    1   1       0   1   0  ...    True   
499  65.0    1  28.04     2  53.0    1   1       1   1   0  ...   False   

      MR_2   MR_3   MR_4  Thrombolysis_1  Thrombolysis_2  Thrombolysis_3  \
0    False  False  Fals

In [ ]:
# Splitting data into train and test sets
X = data.drop(columns=['HF'])  # Target variable
y = data['HF']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Use Robust Scaling: Instead of standard scaling, use sklearn.preprocessing.RobustScaler() to reduce the effect of outliers.

In [ ]:
from sklearn.preprocessing import RobustScaler

# Initialize RobustScaler
scaler = RobustScaler()

# Fit and transform training data
X_train = scaler.fit_transform(X_train)

# Transform test data (avoid data leakage by not fitting on test data)
X_test = scaler.transform(X_test)


In [ ]:
# Convert the NumPy array back to a pandas DataFrame
X_train = pd.DataFrame(X_train, columns=X.columns) # Assuming X was your original DataFrame

# Now you can use the head() method
X_train.describe(include=[np.number]).T

,count,mean,std,min,25%,50%,75%,max
Age,400.0,0.020882,0.712556,-1.882353,-0.470588,0.0,0.529412,1.823529
Sex,400.0,-0.287500,0.453163,-1.000000,-1.000000,0.0,0.000000,0.000000
BMI,400.0,0.118801,0.852797,-2.065910,-0.420854,0.0,0.579146,5.205835
NYHA,400.0,0.001250,0.449866,-0.500000,-0.500000,0.0,0.500000,1.000000
HR,400.0,0.153281,0.940222,-3.000000,-0.375000,0.0,0.625000,4.687500
HTN,400.0,-0.355000,0.479113,-1.000000,-1.000000,0.0,0.000000,0.000000
DM,400.0,0.470000,0.499724,0.000000,0.000000,0.0,1.000000,1.000000
Smoker,400.0,0.352500,0.478347,0.000000,0.000000,0.0,1.000000,1.000000
DL,400.0,-0.427500,0.495335,-1.000000,-1.000000,0.0,0.000000,0.000000
BA,400.0,0.082500,0.275470,0.000000,0.000000,0.0,0.000000,1.000000


Evaluate models and store results in a DataFrame

In [ ]:
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
)
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder

# Set global seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Define models with fixed random_state
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

models = {
    # 🔹 Logistic Regression: Strong regularization for better generalization
    'Logistic Regression': LogisticRegression(
        C=0.3, solver='liblinear', penalty='l1', class_weight='balanced', random_state=SEED
    ),

    # 🔹 K-Nearest Neighbors: Reducing overfitting with distance-based weighting
    'K-Nearest Neighbors': KNeighborsClassifier(
        n_neighbors=5, weights='distance', algorithm='ball_tree', leaf_size=20, p=2
    ),

    # 🔹 Naive Bayes: Adjusted for better probability estimation
    'Naive Bayes': GaussianNB(var_smoothing=1e-8),

    # 🔹 Decision Tree: More depth and splitting to capture complex patterns
    'Decision Tree': DecisionTreeClassifier(
        criterion='entropy', max_depth=20, min_samples_split=3, min_samples_leaf=2, random_state=SEED
    ),

    # 🔹 Random Forest: More trees & depth for higher accuracy
    'Random Forest': RandomForestClassifier(
        n_estimators=300, max_depth=15, min_samples_split=3, min_samples_leaf=1,
        class_weight='balanced', random_state=SEED
    ),

    # 🔹 Support Vector Machine: Higher C, balanced class weights
    'Support Vector Machine': SVC(
        C=5.0, kernel='rbf', gamma='scale', probability=True, class_weight='balanced', random_state=SEED
    ),

    # 🔹 Ridge Classifier: Optimized regularization for stability
    'Ridge Classifier': RidgeClassifier(alpha=0.3, class_weight='balanced'),

    # 🔹 Quadratic Discriminant Analysis: Optimized for stability
     #'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis(reg_param=0.03),

    # 🔹 Linear Discriminant Analysis: Optimized shrinkage
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto'),

    # 🔹 AdaBoost: More estimators & lower learning rate
    'AdaBoost': AdaBoostClassifier(
        n_estimators=200, learning_rate=0.05, random_state=SEED
    ),

    # 🔹 Gradient Boosting: Lower learning rate, more estimators
    'Gradient Boosting': GradientBoostingClassifier(
        learning_rate=0.03, n_estimators=250, max_depth=10, min_samples_split=3,
        min_samples_leaf=2, subsample=0.85, random_state=SEED
    ),

    # 🔹 Extra Trees Classifier: Higher estimators for robustness
    'Extra Trees Classifier': ExtraTreesClassifier(
        n_estimators=300, max_depth=12, min_samples_split=4, random_state=SEED
    ),

    # 🔹 LightGBM: Optimized for best recall & precision
    'LightGBM': lgb.LGBMClassifier(
        colsample_bytree=0.9, learning_rate=0.03, max_depth=20, min_child_samples=5,
        n_estimators=250, num_leaves=90, subsample=0.85, class_weight='balanced',verbose=-1,random_state=SEED
    ),

    # 🔹 Multi-Layer Perceptron (MLP): More layers & epochs for better feature learning
    'Multi-Layer Perceptron (MLP)': MLPClassifier(
        hidden_layer_sizes=(256, 128, 64), activation='relu', solver='adam', alpha=0.0001,
        batch_size=16, learning_rate='adaptive', max_iter=500, random_state=SEED
    ),

    # 🔹 XGBoost: Tuned for high performance
    'XGBoost': XGBClassifier(
        colsample_bytree=0.8, learning_rate=0.03, max_depth=12, min_child_weight=4,
        n_estimators=250, subsample=0.85, gamma=0.1, reg_alpha=0.1, reg_lambda=0.1,
        scale_pos_weight=1, random_state=SEED
    ),

    # 🔹 CatBoost: Tuned for medical applications
    'CatBoost': CatBoostClassifier(
        iterations=250, learning_rate=0.03, depth=14, min_data_in_leaf=5,
        subsample=0.85, l2_leaf_reg=3, class_weights=[1, 4], random_state=SEED, verbose=0
    )
}

# Evaluate models and store results in a DataFrame
def evaluate_models(models, X_train, X_test, y_train, y_test):
    results = []
    for name, model in models.items():
        # Fit the model to the training data
        model.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Convert y_test and y_pred to integers before calculating accuracy
        accuracy = accuracy_score(y_test.astype(int), y_pred.astype(int))

        # Get classification report for multi-class classification
        report = classification_report(y_test.astype(int), y_pred.astype(int), output_dict=True)

        # Collect results dynamically for each class
        model_result = {
            'Model': name,
            'Accuracy': accuracy,
        }

        # Add Precision, Recall, F1 score for each class dynamically
        for label in report.keys():
            if label != 'accuracy':  # Exclude the accuracy key
                model_result[f'Precision (Class {label})'] = report[label]['precision']
                model_result[f'Recall (Class {label})'] = report[label]['recall']
                model_result[f'F1 Score (Class {label})'] = report[label]['f1-score']

        results.append(model_result)

    # Create a DataFrame from results
    results_df = pd.DataFrame(results)
    return results_df

# Run evaluation with models
# Convert target variable to integers
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Run evaluation with models
results_df = evaluate_models(models, X_train, X_test, y_train, y_test)

# Sort results by accuracy
sorted_results_df = results_df.sort_values(by='Accuracy', ascending=False)

# Display the sorted results
print("\nSorted Results by Accuracy:")
print(sorted_results_df)

# Print the best three models
print("\nTop 3 Best Models Based on Overall Performance:\n")
print(sorted_results_df.head(3))

# Store best models
best_models = sorted_results_df.head(3)


Sorted Results by Accuracy:
                           Model  Accuracy  Precision (Class 0)  \
11                      LightGBM      0.84             0.853659   
4                  Random Forest      0.82             0.846154   
6               Ridge Classifier      0.82             0.829268   
9              Gradient Boosting      0.82             0.846154   
10        Extra Trees Classifier      0.82             0.829268   
14                      CatBoost      0.82             0.935484   
13                       XGBoost      0.81             0.795455   
0            Logistic Regression      0.80             0.804878   
12  Multi-Layer Perceptron (MLP)      0.80             0.804878   
7   Linear Discriminant Analysis      0.79             0.815789   
3                  Decision Tree      0.77             0.750000   
5         Support Vector Machine      0.77             0.761905   
2                    Naive Bayes      0.76             0.783784   
8                       AdaBoost 

Function to create a comparison plot with line graphs for all metrics in one plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to create a comparison plot with line graphs for all metrics in one plot
def plot_model_comparison_all_metrics(sorted_results_df):
    # Set the plot style
    sns.set(style="whitegrid")

    # Prepare the data for plotting
    metrics = [
        'Accuracy',
        'Precision (Class macro avg)',
        'Recall (Class macro avg)',
        'F1 Score (Class macro avg)',
        'Precision (Class weighted avg)',
        'Recall (Class weighted avg)',
        'F1 Score (Class weighted avg)'
    ]

    # Create the figure and axis for plotting
    plt.figure(figsize=(14, 8))  # Adjusted plot size

    # Define a color palette for distinct line colors
    color_palette = sns.color_palette("Set2", len(metrics))  # Choose a different color for each metric

    # Iterate through each metric and plot its value
    for i, metric in enumerate(metrics):
        sns.lineplot(x='Model', y=metric, data=sorted_results_df, label=metric,
                     marker='o', linewidth=2, color=color_palette[i])

    # Set the title and labels with larger fonts
    plt.title('Comparison of Metrics for Each Model (Before Cross-Validation and Without Feature Selection)', fontsize=18)
    plt.xlabel('Model', fontsize=16)
    plt.ylabel('Score', fontsize=16)
    plt.xticks(rotation=90, fontsize=14)
    plt.yticks(fontsize=14)

    # Add legend
    plt.legend(title='Metrics', fontsize=12)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Example usage
plot_model_comparison_all_metrics(sorted_results_df)


## Features Selections part


### Selected Important Features Based on Variance Threshold

Recall data


In [ ]:
# Drop unnecessary columns
#dataf =dataf.drop(columns=['StudyID'])
# Splitting data into train and test sets
X = dataf.drop(columns=['HF'])  # Target variable
y = dataf['HF']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
print(dataf.columns)

Index(['Age', 'Sex', 'BMI', 'NYHA', 'HR', 'HTN', 'DM', 'Smoker', 'DL', 'BA',
       'RBS', 'HbA1C', 'Creatinine', 'Na', 'K', 'Cl', 'Hb', 'TropI', 'CXR',
       'ECG', 'LVIDd', 'FS', 'LVIDs', 'LVEF', 'RWMA', 'LAV', 'MI', 'ACS',
       'Wall', 'Thrombolysis', 'ICT', 'IRT', 'MR', 'EA', 'DT', 'MPI', 'RR',
       'Chest_pain', 'TC', 'LDLc', 'HDLc', 'TG', 'BNP', 'HF'],
      dtype='object')


In [ ]:
# Exclude the target column from numeric columns
numeric_cols = [col for col in X_train.columns if col != 'HF_1']

# Scaling numerical features (avoiding data leakage)
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Convert HF to numeric (if not already)
dataf['HF'] = pd.to_numeric(dataf['HF'])

# Compute correlation matrix
correlation_matrix = dataf.corr()

# Extract correlations with HF
hf_correlation = correlation_matrix['HF'].sort_values(ascending=False)

# Display correlations
print("Correlation of numerical features with HF:\n", hf_correlation)

# Plot heatmap for better visualization
plt.figure(figsize=(50, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()


Correlation of numerical features with HF:
 HF              1.000000
RWMA            0.555569
NYHA            0.547427
MI              0.523669
Chest_pain      0.517996
LVIDs           0.487436
Wall            0.412381
ACS             0.404052
ECG             0.402677
LVIDd           0.396085
CXR             0.349754
MR              0.300189
Age             0.218864
TropI           0.213556
BNP             0.204609
IRT             0.204211
HR              0.201229
RBS             0.190930
RR              0.158264
DM              0.148409
HTN             0.148131
TC              0.144203
DL              0.141524
Creatinine      0.126285
Sex             0.123122
LDLc            0.122968
Smoker          0.111850
Thrombolysis    0.110787
ICT             0.110719
MPI             0.099581
HbA1C           0.075143
LAV             0.050025
TG              0.049793
HDLc            0.033168
K               0.010638
Cl              0.004967
EA             -0.007328
Na             -0.031671
Hb    

In [ ]:
import pandas as pd

# Split the dataset into HF=1 (Heart Failure) and HF=0 (No Heart Failure)
hf_1 = dataf[dataf['HF'] == 1]
hf_0 = dataf[dataf['HF'] == 0] # Use 'HF_1' and check for value 0

# ... (rest of the code)

# Compute mean difference for each numerical variable
numerical_features = dataf.select_dtypes(include=['float64']).columns
mean_diff = pd.DataFrame({
    'Mean (HF=1)': hf_1[numerical_features].mean(),
    'Mean (HF=0)': hf_0[numerical_features].mean(),
    'Difference': hf_1[numerical_features].mean() - hf_0[numerical_features].mean()
})

# Sort features by difference (absolute value)
mean_diff = mean_diff.sort_values(by='Difference', ascending=False)

# Display top features with highest differences
print(mean_diff.head(44))

# Plot the top features with highest differences
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.barplot(x=mean_diff['Difference'].head(44), y=mean_diff.index[:44], palette='viridis')
plt.axvline(x=0, color='black', linestyle='--')  # Reference line at zero
plt.title('Top Numerical Features Differentiating HF=1 vs HF=0')
plt.xlabel('Mean Difference (HF=1 - HF=0)')
plt.ylabel('Features')
plt.show()

            Mean (HF=1)  Mean (HF=0)  Difference
BNP          124.700730    40.225664   84.475066
TC           181.211679   167.694690   13.516989
TG           175.288321   165.283186   10.005135
LDLc         117.620438   108.327434    9.293004
IRT          101.799270    93.473451    8.325819
LVIDs         39.580292    31.283186    8.297106
TropI         15.375964     7.830193    7.545771
HR            83.430657    77.402655    6.028002
LVIDd         51.452555    45.575221    5.877334
Age           58.051095    52.473451    5.577644
ICT           89.675182    85.827434    3.847749
RBS            8.840474     7.484956    1.355519
RR            20.952555    19.911504    1.041050
LAV           35.529197    34.615044    0.914153
HDLc          34.740876    34.331858    0.409018
Creatinine     1.478796     1.154735    0.324061
HbA1C          6.395255     6.128319    0.266937
Cl           101.631387   101.579646    0.051741
MPI            0.234745     0.199602    0.035143
K              3.918

In [ ]:
import pandas as pd

# Split the dataset into HF=1 (Heart Failure) and HF=0 (No Heart Failure)
hf_1 = dataf[dataf['HF'] == 1]
hf_0 = dataf[dataf['HF'] == 0] # Use 'HF_1' and check for value 0

# ... (rest of the code)

# Compute mean difference for each numerical variable
numerical_features = dataf.select_dtypes(include=['float64']).columns
mean_diff = pd.DataFrame({
    'Mean (HF=1)': hf_1[numerical_features].mean(),
    'Mean (HF=0)': hf_0[numerical_features].mean(),
    'Difference': hf_1[numerical_features].mean() - hf_0[numerical_features].mean()
})

# Sort features by difference (absolute value)
mean_diff = mean_diff.sort_values(by='Difference', ascending=False)

# Display top features with highest differences
print(mean_diff.head(25))

# Plot the top features with highest differences
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.barplot(x=mean_diff['Difference'].head(10), y=mean_diff.index[:10], palette='viridis')
plt.axvline(x=0, color='black', linestyle='--')  # Reference line at zero
plt.title('Top Numerical Features Differentiating HF=1 vs HF=0')
plt.xlabel('Mean Difference (HF=1 - HF=0)')
plt.ylabel('Features')
plt.show()

            Mean (HF=1)  Mean (HF=0)  Difference
BNP          124.700730    40.225664   84.475066
TC           181.211679   167.694690   13.516989
TG           175.288321   165.283186   10.005135
LDLc         117.620438   108.327434    9.293004
IRT          101.799270    93.473451    8.325819
LVIDs         39.580292    31.283186    8.297106
TropI         15.375964     7.830193    7.545771
HR            83.430657    77.402655    6.028002
LVIDd         51.452555    45.575221    5.877334
Age           58.051095    52.473451    5.577644
ICT           89.675182    85.827434    3.847749
RBS            8.840474     7.484956    1.355519
RR            20.952555    19.911504    1.041050
LAV           35.529197    34.615044    0.914153
HDLc          34.740876    34.331858    0.409018
Creatinine     1.478796     1.154735    0.324061
HbA1C          6.395255     6.128319    0.266937
Cl           101.631387   101.579646    0.051741
MPI            0.234745     0.199602    0.035143
K              3.918

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have your DataFrame 'dataf'

# Split the dataset into HF=1 (Heart Failure) and HF=0 (No Heart Failure)
hf_1 = dataf[dataf['HF'] == 1]
hf_0 = dataf[dataf['HF'] == 0]  # Use 'HF_1' and check for value 0

# ... (rest of the code)

# Compute mean difference for each numerical variable
numerical_features = dataf.select_dtypes(include=['float64']).columns
num_diff = pd.DataFrame({  # Assign the DataFrame to num_diff here
    'Mean (HF=1)': hf_1[numerical_features].mean(),
    'Mean (HF=0)': hf_0[numerical_features].mean(),
    'Difference': hf_1[numerical_features].mean() - hf_0[numerical_features].mean()
})

# Sort by absolute difference (strongest differences at the top)
num_diff = num_diff.sort_values(by="Difference", ascending=False)

# Display the full numeric variable comparison
print(num_diff)

            Mean (HF=1)  Mean (HF=0)  Difference
BNP          124.700730    40.225664   84.475066
TC           181.211679   167.694690   13.516989
TG           175.288321   165.283186   10.005135
LDLc         117.620438   108.327434    9.293004
IRT          101.799270    93.473451    8.325819
LVIDs         39.580292    31.283186    8.297106
TropI         15.375964     7.830193    7.545771
HR            83.430657    77.402655    6.028002
LVIDd         51.452555    45.575221    5.877334
Age           58.051095    52.473451    5.577644
ICT           89.675182    85.827434    3.847749
RBS            8.840474     7.484956    1.355519
RR            20.952555    19.911504    1.041050
LAV           35.529197    34.615044    0.914153
HDLc          34.740876    34.331858    0.409018
Creatinine     1.478796     1.154735    0.324061
HbA1C          6.395255     6.128319    0.266937
Cl           101.631387   101.579646    0.051741
MPI            0.234745     0.199602    0.035143
K              3.918

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming 'data' is your original dataset, and 'selected_features' is already defined
# Also assuming 'feature_variances' is already calculated

# Calculate the variance for each feature in the dataset
# Select only numeric columns before calculating variance
feature_variances = pd.DataFrame({'Feature': dataf.select_dtypes(include=[np.number]).columns,
                                   'Variance': dataf.select_dtypes(include=[np.number]).var()})

# Apply the variance threshold (for example, threshold = 0.80)
threshold = 0.50
selected_var = feature_variances[feature_variances['Variance'] >= threshold]

# Show selected important features based on the variance threshold
print("\nSelected Important Features Based on Variance Threshold:")
print(list(selected_var['Feature']))

# Create a bar plot of the selected features based on their variance
plt.figure(figsize=(12, 6))
sns.barplot(x="Variance", y="Feature", data=selected_var, palette="Blues")

# Labeling the plot
plt.xlabel("Variance")
plt.ylabel("Feature Name")
plt.title(f"Selected Features Based on Variance Threshold ({threshold})")
plt.tight_layout()  # Adjust layout for better spacing
plt.show()


Selected Important Features Based on Variance Threshold:
['Age', 'BMI', 'NYHA', 'HR', 'RBS', 'HbA1C', 'Creatinine', 'Na', 'Cl', 'Hb', 'TropI', 'LVIDd', 'FS', 'LVIDs', 'LVEF', 'LAV', 'ICT', 'IRT', 'DT', 'RR', 'TC', 'LDLc', 'HDLc', 'TG', 'BNP']


### SHAP analysis for important Features

In [ ]:
%pip install shap

  Using cached slicer-0.0.8-py3-none-any.whl.metadata (4.0 kB)
Using cached slicer-0.0.8-py3-none-any.whl (15 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import shap
import pandas as pd
import matplotlib.pyplot as plt

# Get feature names directly from the model
feature_names = models['LightGBM'].feature_names_in_

# Create a DataFrame for SHAP values
X_test_df = pd.DataFrame(X_test, columns=feature_names)

# Create the SHAP explainer
explainer = shap.Explainer(models['LightGBM'])

# Calculate SHAP values
shap_values = explainer(X_test_df)
# Generate the summary plot for top 30 features
shap.summary_plot(shap_values[:, :37], X_test_df.iloc[:, :37])


In [ ]:

# shap.importance_plot(shap_values, max_display=30)  # Line causing the error
shap.plots.bar(shap_values, max_display=37)  # Use shap.plots.bar instead

In [ ]:
data.columns

Index(['Age', 'Sex', 'BMI', 'NYHA', 'HR', 'HTN', 'DM', 'Smoker', 'DL', 'BA',
       'RBS', 'HbA1C', 'Creatinine', 'Na', 'K', 'Cl', 'Hb', 'TropI', 'CXR',
       'ECG', 'LVIDd', 'FS', 'LVIDs', 'LVEF', 'RWMA', 'LAV', 'MI', 'ACS',
       'Wall', 'Thrombolysis', 'ICT', 'IRT', 'MR', 'EA', 'DT', 'MPI', 'RR',
       'Chest_pain', 'TC', 'LDLc', 'HDLc', 'TG', 'BNP', 'HF'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 44 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           500 non-null    float64
 1   Sex           500 non-null    int64  
 2   BMI           500 non-null    float64
 3   NYHA          500 non-null    int64  
 4   HR            500 non-null    float64
 5   HTN           500 non-null    int64  
 6   DM            500 non-null    int64  
 7   Smoker        500 non-null    int64  
 8   DL            500 non-null    int64  
 9   BA            500 non-null    int64  
 10  RBS           500 non-null    float64
 11  HbA1C         500 non-null    float64
 12  Creatinine    500 non-null    float64
 13  Na            500 non-null    float64
 14  K             500 non-null    float64
 15  Cl            500 non-null    float64
 16  Hb            500 non-null    float64
 17  TropI         500 non-null    float64
 18  CXR           500 non-null    

## Features to drop by Based on Variance Threshold ,SHAP analysis and  Domain Expert and create a new data set


In [ ]:
import pandas as pd

# Convert X_train and X_test to DataFrames if they are NumPy arrays
if isinstance(X_train, np.ndarray):
    X_train = pd.DataFrame(X_train, columns=feature_names)

if isinstance(X_test, np.ndarray):
    X_test = pd.DataFrame(X_test, columns=feature_names)

# List of features to drop
features_to_drop = ["BA", "HbA1C", "Na", "K", "Cl", "Hb", "MPI", "HDLc"]

# Drop features from the dataset
X_train.drop(columns=features_to_drop, inplace=True, errors='ignore')  # Ignore errors if feature not found
X_test.drop(columns=features_to_drop, inplace=True, errors='ignore')

print("Dropped features successfully. New shape of dataset:")
print(f"Training Data: {X_train.shape}, Testing Data: {X_test.shape}")


Dropped features successfully. New shape of dataset:
Training Data: (400, 35), Testing Data: (100, 35)


### Evaluate models and store results to find best 3 Models

In [ ]:
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
)
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder

# Set global seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Define models with fixed random_state
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

models = {
    # 🔹 Logistic Regression: Strong regularization for better generalization
    'Logistic Regression': LogisticRegression(
        C=0.3, solver='liblinear', penalty='l1', class_weight='balanced', random_state=SEED
    ),

    # 🔹 K-Nearest Neighbors: Reducing overfitting with distance-based weighting
    'K-Nearest Neighbors': KNeighborsClassifier(
        n_neighbors=5, weights='distance', algorithm='ball_tree', leaf_size=20, p=2
    ),

    # 🔹 Naive Bayes: Adjusted for better probability estimation
    'Naive Bayes': GaussianNB(var_smoothing=1e-8),

    # 🔹 Decision Tree: More depth and splitting to capture complex patterns
    'Decision Tree': DecisionTreeClassifier(
        criterion='entropy', max_depth=20, min_samples_split=3, min_samples_leaf=2, random_state=SEED
    ),

    # 🔹 Random Forest: More trees & depth for higher accuracy
    'Random Forest': RandomForestClassifier(
        n_estimators=300, max_depth=15, min_samples_split=3, min_samples_leaf=1,
        class_weight='balanced', random_state=SEED
    ),

    # 🔹 Support Vector Machine: Higher C, balanced class weights
    'Support Vector Machine': SVC(
        C=5.0, kernel='rbf', gamma='scale', probability=True, class_weight='balanced', random_state=SEED
    ),

    # 🔹 Ridge Classifier: Optimized regularization for stability
    'Ridge Classifier': RidgeClassifier(alpha=0.3, class_weight='balanced'),

    # 🔹 Quadratic Discriminant Analysis: Optimized for stability
    #'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis(reg_param=0.03),

    # 🔹 Linear Discriminant Analysis: Optimized shrinkage
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto'),

    # 🔹 AdaBoost: More estimators & lower learning rate
    'AdaBoost': AdaBoostClassifier(
        n_estimators=200, learning_rate=0.05, random_state=SEED
    ),

    # 🔹 Gradient Boosting: Lower learning rate, more estimators
    'Gradient Boosting': GradientBoostingClassifier(
        learning_rate=0.03, n_estimators=250, max_depth=10, min_samples_split=3,
        min_samples_leaf=2, subsample=0.85, random_state=SEED
    ),

    # 🔹 Extra Trees Classifier: Higher estimators for robustness
    'Extra Trees Classifier': ExtraTreesClassifier(
        n_estimators=300, max_depth=12, min_samples_split=4, random_state=SEED
    ),

    # 🔹 LightGBM: Optimized for best recall & precision
    'LightGBM': lgb.LGBMClassifier(
        colsample_bytree=0.9, learning_rate=0.03, max_depth=20, min_child_samples=5,
        n_estimators=250, num_leaves=90, subsample=0.85, class_weight='balanced',verbose=-1, random_state=SEED
    ),

    # 🔹 Multi-Layer Perceptron (MLP): More layers & epochs for better feature learning
    'Multi-Layer Perceptron (MLP)': MLPClassifier(
        hidden_layer_sizes=(256, 128, 64), activation='relu', solver='adam', alpha=0.0001,
        batch_size=16, learning_rate='adaptive', max_iter=500, random_state=SEED
    ),

    # 🔹 XGBoost: Tuned for high performance
    'XGBoost': XGBClassifier(
        colsample_bytree=0.8, learning_rate=0.03, max_depth=12, min_child_weight=4,
        n_estimators=250, subsample=0.85, gamma=0.1, reg_alpha=0.1, reg_lambda=0.1,
        scale_pos_weight=1, random_state=SEED
    ),

    # 🔹 CatBoost: Tuned for medical applications
    'CatBoost': CatBoostClassifier(
        iterations=250, learning_rate=0.03, depth=14, min_data_in_leaf=5,
        subsample=0.85, l2_leaf_reg=3, class_weights=[1, 4], random_state=SEED, verbose=0
    )
}

# Evaluate models and store results in a DataFrame
def evaluate_models(models, X_train, X_test, y_train, y_test):
    results = []
    for name, model in models.items():
        # Fit the model to the training data
        model.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Convert y_test and y_pred to integers before calculating accuracy
        accuracy = accuracy_score(y_test.astype(int), y_pred.astype(int))

        # Get classification report for multi-class classification
        report = classification_report(y_test.astype(int), y_pred.astype(int), output_dict=True)

        # Collect results dynamically for each class
        model_result = {
            'Model': name,
            'Accuracy': accuracy,
        }

        # Add Precision, Recall, F1 score for each class dynamically
        for label in report.keys():
            if label != 'accuracy':  # Exclude the accuracy key
                model_result[f'Precision (Class {label})'] = report[label]['precision']
                model_result[f'Recall (Class {label})'] = report[label]['recall']
                model_result[f'F1 Score (Class {label})'] = report[label]['f1-score']

        results.append(model_result)

    # Create a DataFrame from results
    results_df = pd.DataFrame(results)
    return results_df

# Run evaluation with models
results_df = evaluate_models(models, X_train, X_test, y_train, y_test)

# Sort results by accuracy
sorted_results_df = results_df.sort_values(by='Accuracy', ascending=False)

# Display the sorted results
print("\nSorted Results by Accuracy:")
print(sorted_results_df)

# Print the best three models
print("\nTop 3 Best Models Based on Overall Performance:\n")
print(sorted_results_df.head(15))

# Store best models
best_models = sorted_results_df.head(15)


Sorted Results by Accuracy:
                           Model  Accuracy  Precision (Class 0)  \
11                      LightGBM      0.85             0.875000   
4                  Random Forest      0.84             0.853659   
10        Extra Trees Classifier      0.84             0.871795   
6               Ridge Classifier      0.83             0.850000   
9              Gradient Boosting      0.83             0.850000   
7   Linear Discriminant Analysis      0.81             0.842105   
13                       XGBoost      0.81             0.825000   
14                      CatBoost      0.81             0.933333   
0            Logistic Regression      0.79             0.785714   
3                  Decision Tree      0.79             0.785714   
5         Support Vector Machine      0.79             0.815789   
12  Multi-Layer Perceptron (MLP)      0.79             0.800000   
2                    Naive Bayes      0.76             0.783784   
8                       AdaBoost 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to create a comparison plot with line graphs for all metrics in one plot
def plot_model_comparison_all_metrics(sorted_results_df):
    # Set the plot style
    sns.set(style="whitegrid")

    # Prepare the data for plotting
    metrics = [
        'Accuracy',
        'Precision (Class macro avg)',
        'Recall (Class macro avg)',
        'F1 Score (Class macro avg)',
        'Precision (Class weighted avg)',
        'Recall (Class weighted avg)',
        'F1 Score (Class weighted avg)'
    ]

    # Create the figure and axis for plotting
    plt.figure(figsize=(14, 8))  # Adjusted plot size

    # Define a color palette for distinct line colors
    color_palette = sns.color_palette("Set2", len(metrics))  # Choose a different color for each metric

    # Iterate through each metric and plot its value
    for i, metric in enumerate(metrics):
        sns.lineplot(x='Model', y=metric, data=sorted_results_df, label=metric,
                     marker='o', linewidth=2, color=color_palette[i])

    # Set the title and labels with larger fonts
    plt.title('Comparison of Metrics for Each Model (Before Cross-Validation and Hyperamiter ,With Feature Selection)', fontsize=18)
    plt.xlabel('Model', fontsize=16)
    plt.ylabel('Score', fontsize=16)
    plt.xticks(rotation=90, fontsize=14)
    plt.yticks(fontsize=14)

    # Add legend
    plt.legend(title='Metrics', fontsize=12)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Example usage
plot_model_comparison_all_metrics(sorted_results_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to create a bar chart comparing accuracy
def plot_accuracy_comparison(sorted_results_df):
    # Set the plot style
    sns.set(style="whitegrid")

    # Create the figure and axis for plotting
    plt.figure(figsize=(12, 6))

    # Create the bar plot for accuracy
    sns.barplot(x='Model', y='Accuracy', data=sorted_results_df, palette='Blues_r')

    # Set the title and labels
    plt.title('Accuracy Comparison of Models', fontsize=16)
    plt.xlabel('Model', fontsize=14)
    plt.ylabel('Accuracy Score', fontsize=14)
    plt.xticks(rotation=90, fontsize=12)
    plt.yticks(fontsize=12)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Example usage
plot_accuracy_comparison(sorted_results_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to create a bar chart comparing accuracy with percentage labels
def plot_accuracy_comparison(sorted_results_df):
    # Set the plot style
    sns.set(style="whitegrid")

    # Create the figure and axis for plotting
    plt.figure(figsize=(20, 12))

    # Create the bar plot for accuracy
    ax = sns.barplot(x='Model', y='Accuracy', data=sorted_results_df, palette='Blues_r')

    # Add percentage labels on top of the bars
    for p in ax.patches:
        ax.annotate(f'{p.get_height()*100:.1f}%',  # Convert to percentage
                    (p.get_x() + p.get_width() / 2, p.get_height()),
                    ha='center', va='bottom', fontsize=12, fontweight='bold')

    # Set the title and labels
    plt.title('Accuracy Comparison of Models', fontsize=16)
    plt.xlabel('Model', fontsize=14)
    plt.ylabel('Accuracy Score', fontsize=14)
    plt.xticks(rotation=90, fontsize=12)
    plt.yticks(fontsize=12)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Example usage
plot_accuracy_comparison(sorted_results_df)


#### Confusion matrix for the best 3 models

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Function to plot confusion matrix for the best models
def plot_confusion_matrices(models, X_train, y_train, X_test, y_test, best_models):
    for name in best_models['Model']:
        model = models[name]

        # Fit the model and make predictions
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Generate the confusion matrix
        cm = confusion_matrix(y_test.astype(int), y_pred.astype(int))  # Convert to int before calculating

        # Plot the confusion matrix
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Class 0", "Class 1"])
        disp.plot(cmap=plt.cm.Blues)

        plt.title(f"Confusion Matrix for {name}")
        plt.show()

# Call the function to plot confusion matrices for the top 3 models
plot_confusion_matrices(models, X_train, y_train, X_test, y_test, best_models)

### ROC Curve and calculate AUC for the best 3 models

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Function to plot ROC Curve and calculate AUC for the best models
def plot_roc_auc(models, X_train, y_train, X_test, y_test, best_models):
    plt.figure(figsize=(10, 8))

    for name in best_models['Model']:  # Assuming best_models is a DataFrame with a 'Model' column
        model = models[name]

        # Fit the model and get probabilities for ROC curve
        model.fit(X_train, y_train)

        # Check if the model has predict_proba method before calling it
        if hasattr(model, 'predict_proba'):
            y_prob = model.predict_proba(X_test)[:, 1]  # Get probabilities for class 1
        else:
            # For models without predict_proba, use decision_function if available
            if hasattr(model, 'decision_function'):
                y_prob = model.decision_function(X_test)
            else:
                # If neither predict_proba nor decision_function is available, skip the model
                print(f"Skipping ROC curve for {name} as it does not have predict_proba or decision_function")
                continue

        # Compute ROC curve and AUC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)

        # Plot ROC curve
        plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.4f})')

    # Plot the random classifier (diagonal line)
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')

    # Labels and title
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
    plt.legend(loc='lower right', fontsize=12)
    plt.grid(True)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Example usage
plot_roc_auc(models, X_train, y_train, X_test, y_test, best_models)

## Optimizing Multiple ML Models : Optuna


In [ ]:
import optuna
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['Extra Trees Classifier', 'RandomForest', 'LightGBM'])

    if classifier_name == 'Extra Trees Classifier':
        # Extra Trees Classifier hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 1000, 2000)
        max_depth = trial.suggest_int('max_depth', 10, 30)
        min_samples_split = trial.suggest_int('min_samples_split', 10, 30)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

        model = ExtraTreesClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 100, 2000)
        max_depth = trial.suggest_int('max_depth', 10, 30)
        min_samples_split = trial.suggest_int('min_samples_split', 5, 20)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'LightGBM':
        # LightGBM hyperparameters
        boosting_type = trial.suggest_categorical('boosting_type', ['dart'])
        n_estimators = trial.suggest_int('n_estimators', 100, 2000)
        learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.001)
        max_bin = trial.suggest_int('max_bin', 500, 2000)
        importance_type = trial.suggest_categorical('importance_type', ['gain'])

        model = LGBMClassifier(
            boosting_type=boosting_type,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_bin=max_bin,
            importance_type=importance_type,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy').mean()
    return score


In [ ]:
pip install optuna

  Using cached optuna-4.2.1-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.15.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached Mako-1.3.9-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.2.1-py3-none-any.whl (383 kB)
Using cached alembic-1.15.1-py3-none-any.whl (231 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.4 MB/s eta 0:00:00a 0:00:01
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
Using cached Mako-1.3.9-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-03-25 15:14:30,829] A new study created in memory with name: no-name-09afa9d3-440a-4237-bc2c-1d6fd7ea8941
[I 2025-03-25 15:14:38,975] Trial 0 finished with value: 0.865 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1656, 'max_depth': 24, 'min_samples_split': 9, 'min_samples_leaf': 3, 'bootstrap': True}. Best is trial 0 with value: 0.865.
[I 2025-03-25 15:14:41,189] Trial 1 finished with value: 0.8525 and parameters: {'classifier': 'RandomForest', 'n_estimators': 487, 'max_depth': 28, 'min_samples_split': 17, 'min_samples_leaf': 12, 'bootstrap': True}. Best is trial 0 with value: 0.865.
[I 2025-03-25 15:14:46,285] Trial 2 finished with value: 0.8474999999999999 and parameters: {'classifier': 'Extra Trees Classifier', 'n_estimators': 1619, 'max_depth': 18, 'min_samples_split': 18, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.865.
[I 2025-03-25 15:14:49,112] Trial 3 finished with value: 0.7849999999999999 and parameters: {'classifier': 'LightGBM', 'boosti

In [ ]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

In [ ]:
study.trials_dataframe()['params_classifier'].value_counts()

In [ ]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

In [ ]:
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_slice,
    plot_param_importances
)
# Check if the study contains completed trials before plotting
if len(study.trials) > 0:
    # Plot the optimization history
    plot_optimization_history(study).show()

    # Plot the slice plot
    plot_slice(study).show()

    # Plot hyperparameter importance
    plot_param_importances(study).show()
else:
    print("No completed trials available for visualization.")

# Random Forest

False, 'class_weight': 'balanced', 'max_depth': 15, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300}

### Train Random Forest Model


In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Define Random Forest Model with Given Hyperparameters
rf_model = RandomForestClassifier(
    n_estimators=1180, max_depth=17, min_samples_split=9, min_samples_leaf=1, bootstrap=False
)

# Fit Model
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")
# Fit Model
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get feature importances
importances = rf_model.feature_importances_

# Sort the features by importance in descending order
indices = np.argsort(importances)[::-1]

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importances - Random Forest Model")
plt.bar(range(X_train.shape[1]), importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), np.array(X_train.columns)[indices], rotation=90)
plt.xlabel("Features")
plt.ylabel("Importance")
plt.show()


Print Classification Report and Confusion Matrix

In [ ]:
# Print Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix for Random Forest")
plt.show()


Perform 10-Fold Cross-Validation and Compute Loss Function

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_predict
from lightgbm import LGBMClassifier
rf_clf = RandomForestClassifier(
    n_estimators=1180, max_depth=17, min_samples_split=9, min_samples_leaf=1, bootstrap=False, random_state=42

)



# Perform 10-fold cross-validation
cv_scores = cross_val_score(rf_clf, X_train, y_train,
                            cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
                            scoring='accuracy')

# Assuming loss_values for each fold (you would replace this with actual loss values if available)
# Example: Loss values for each fold (you might need to calculate these)
loss_values = [1 - score for score in cv_scores]  # Just an example, assuming loss is 1 - accuracy

# Accuracy Before Cross-validation (Example value, replace with your actual value)
test_accuracy = 0.84  # Example value #########################################################################################################################################

# Cross-validation accuracy
cv_accuracy = np.mean(cv_scores)
print(f"cv_accuracy: {cv_accuracy:.4f}")
# Calculate log loss before cross-validation (using test set)
rf_clf.fit(X_train, y_train)
y_pred_prob_before = rf_clf.predict_proba(X_test)  # predicted probabilities
log_loss_before = log_loss(y_test, y_pred_prob_before)

# Calculate log loss after cross-validation (using cross_val_predict)
y_pred_prob_after = cross_val_predict(rf_clf, X_train, y_train, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42), method='predict_proba')
log_loss_after = log_loss(y_train, y_pred_prob_after)

# Create a figure with subplots (2 rows and 2 columns)
fig, axes = plt.subplots(2, 2, figsize=(14, 12))  # Adjusted plot size

# 1. Accuracy per Fold (Bar chart)
axes[0, 0].bar(range(1, 11), cv_scores, color=[
    'SkyBlue', 'Salmon', 'Tomato', 'MediumSeaGreen', 'RoyalBlue',
    'SlateGray', 'Orchid', 'Goldenrod', 'Crimson', 'DodgerBlue'
], edgecolor='black')
for bar in axes[0, 0].bar(range(1, 11), cv_scores, color=[
    'SkyBlue', 'Salmon', 'Tomato', 'MediumSeaGreen', 'RoyalBlue',
    'SlateGray', 'Orchid', 'Goldenrod', 'Crimson', 'DodgerBlue'
], edgecolor='black'):
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width() / 2, height + 0.01, f'{height*100:.2f}%', ha='center', va='bottom', fontsize=10, color='black')
axes[0, 0].set_xlabel('Fold')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_title('Accuracy per Fold (10-Fold Cross-Validation): Random Forest Classifier')
axes[0, 0].set_xticks(range(1, 11))
axes[0, 0].grid(True, axis='y', linestyle='--', alpha=0.7)

# 2. Loss per Fold (Line graph)
axes[0, 1].plot(range(1, 11), loss_values, marker='o', color='purple', label='Loss')
axes[0, 1].set_xlabel('Fold')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_title('Loss per Fold (10-Fold Cross-Validation): Random Forest Classifier')
axes[0, 1].set_xticks(range(1, 11))
axes[0, 1].grid(True)
axes[0, 1].legend()

# 3. Box Plot for Accuracy Before and After Cross-validation
axes[1, 0].boxplot([cv_scores, [test_accuracy] * 10], labels=['After CV', 'Before CV'])
axes[1, 0].set_title('After Cross-validation Accuracy: Random Forest Classifier')
axes[1, 0].set_ylabel('Accuracy')

# 4. Box Plot for Log Loss Before and After Cross-validation
axes[1, 1].boxplot([np.array([log_loss_after] * 10), np.array([log_loss_before] * 10)],
                   labels=['After CV', 'Before CV'])
axes[1, 1].set_title('After Cross-validation Log Loss: Random Forest Classifier')
axes[1, 1].set_ylabel('Log Loss')

# Display the plots
plt.tight_layout()  # Adjust the layout so that the plots do not overlap
plt.show()

Precision-Recall Curve and Learning Curve for best Model

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.model_selection import learning_curve
import numpy as np

# Precision-Recall Curve
y_pred_proba =rf_clf.predict_proba(X_test)[:, 1]  # Probabilities for Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = average_precision_score(y_test, y_pred_proba)

# Learning Curve
train_sizes, train_scores, test_scores = learning_curve(rf_clf, X_train, y_train, cv=10, n_jobs=-1)
train_mean = train_scores.mean(axis=1)
test_mean = test_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
test_std = test_scores.std(axis=1)

# Create a figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Precision-Recall Curve
axes[0].plot(recall, precision, color='green', lw=2, label=f'Precision-Recall Curve (AUC = {pr_auc:.4f})')
axes[0].set_xlabel('Recall')
axes[0].set_ylabel('Precision')
axes[0].set_title('Precision-Recall Curve')
axes[0].legend(loc="lower left")

# Learning Curve
axes[1].plot(train_sizes, train_mean, color='blue', label='Training Score')
axes[1].plot(train_sizes, test_mean, color='green', label='Cross-Validation Score')
axes[1].fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
axes[1].fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='green', alpha=0.2)
axes[1].set_xlabel('Training Size')
axes[1].set_ylabel('Score')
axes[1].set_title('Learning Curve')
axes[1].legend(loc="best")

# Adjust the layout
plt.tight_layout()
plt.show()


## Apply Soft Voting and Hard Voting ensamble method

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score

# Define individual models with best parameters
rf_model = RandomForestClassifier(
    n_estimators=1180, max_depth=17, min_samples_split=9, min_samples_leaf=1, bootstrap=False, random_state=42
)

et_model = ExtraTreesClassifier(
    n_estimators=1097, max_depth=25, min_samples_split=12, min_samples_leaf=2, random_state=42
)

lgbm_model = LGBMClassifier(
    boosting_type='dart', n_estimators=1830, learning_rate=0.000961, max_bin=1568, importance_type='gain', random_state=42
)

# Apply Hard Voting
hard_voting_clf = VotingClassifier(
    estimators=[('RF', rf_model), ('ET', et_model), ('LGBM', lgbm_model)],
    voting='hard'  # Majority voting
)

# Apply Soft Voting
soft_voting_clf = VotingClassifier(
    estimators=[('RF', rf_model), ('ET', et_model), ('LGBM', lgbm_model)],
    voting='soft'  # Probability-based voting
)

# Evaluate performance using cross-validation
hard_voting_score = cross_val_score(hard_voting_clf, X_train, y_train, cv=10, scoring='accuracy').mean()
soft_voting_score = cross_val_score(soft_voting_clf, X_train, y_train, cv=10, scoring='accuracy').mean()

print(f"Hard Voting Accuracy: {hard_voting_score:.4f}")
print(f"Soft Voting Accuracy: {soft_voting_score:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Individual model scores using cross-validation
rf_score = cross_val_score(rf_model, X_train, y_train, cv=10, scoring='accuracy').mean()
et_score = cross_val_score(et_model, X_train, y_train, cv=10, scoring='accuracy').mean()
lgbm_score = cross_val_score(lgbm_model, X_train, y_train, cv=10, scoring='accuracy').mean()

# Accuracy scores
models = ['Random Forest', 'Extra Trees', 'LightGBM', 'Hard Voting', 'Soft Voting']
scores = [rf_score, et_score, lgbm_score, hard_voting_score, soft_voting_score]

# Plot bar chart
plt.figure(figsize=(8, 5))
sns.barplot(x=models, y=scores, palette='viridis')

# Add percentage value labels on top of the bars
for i, v in enumerate(scores):
    plt.text(i, v + 0.002, f"{v * 100:.2f}%", ha='center', fontsize=12)

plt.ylim(min(scores) - 0.02, max(scores) + 0.02)
plt.ylabel('Accuracy')
plt.title('Model Accuracy Comparison')
plt.xticks(rotation=30)
plt.show()


### Bagging (Bootstrap Aggregating)

In [ ]:
from sklearn.ensemble import BaggingClassifier

# Bagging for RandomForest
bagging_rf = BaggingClassifier(
    estimator=rf_model,  # Change 'base_estimator' to 'estimator'
    n_estimators=50,
    random_state=42
)

# Bagging for Extra Trees
bagging_et = BaggingClassifier(
    estimator=et_model,  # Change 'base_estimator' to 'estimator'
    n_estimators=50,
    random_state=42
)

# Bagging for LightGBM
bagging_lgbm = BaggingClassifier(
    estimator=lgbm_model,  # Change 'base_estimator' to 'estimator'
    n_estimators=50,
    random_state=42
)

# Evaluate Bagging performance
bagging_rf_score = cross_val_score(bagging_rf, X_train, y_train, cv=10, scoring='accuracy').mean()
bagging_et_score = cross_val_score(bagging_et, X_train, y_train, cv=10, scoring='accuracy').mean()
bagging_lgbm_score = cross_val_score(bagging_lgbm, X_train, y_train, cv=10, scoring='accuracy').mean()

print(f"Bagging Random Forest Accuracy: {bagging_rf_score:.4f}")
print(f"Bagging Extra Trees Accuracy: {bagging_et_score:.4f}")
print(f"Bagging LightGBM Accuracy: {bagging_lgbm_score:.4f}")

### Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# Boosting for RandomForest using AdaBoost
boosting_rf = AdaBoostClassifier(
    estimator=rf_model,  # Change 'base_estimator' to 'estimator'
    n_estimators=50,
    random_state=42
)

# Boosting for Extra Trees using AdaBoost
boosting_et = AdaBoostClassifier(
    estimator=et_model,  # Change 'base_estimator' to 'estimator'
    n_estimators=50,
    random_state=42
)

# Boosting for LightGBM using XGBoost (can also use LightGBM's own boosting)
boosting_lgbm = XGBClassifier(
    n_estimators=50,
    learning_rate=0.05,
    random_state=42
)

# Evaluate Boosting performance
boosting_rf_score = cross_val_score(boosting_rf, X_train, y_train, cv=10, scoring='accuracy').mean()
boosting_et_score = cross_val_score(boosting_et, X_train, y_train, cv=10, scoring='accuracy').mean()
boosting_lgbm_score = cross_val_score(boosting_lgbm, X_train, y_train, cv=10, scoring='accuracy').mean()

print(f"Boosting Random Forest Accuracy: {boosting_rf_score:.4f}")
print(f"Boosting Extra Trees Accuracy: {boosting_et_score:.4f}")
print(f"Boosting LightGBM Accuracy: {boosting_lgbm_score:.4f}")

### Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Define base models
base_models = [
    ('rf', rf_model),
    ('et', et_model),
    ('lgbm', lgbm_model)
]

# Meta-model (logistic regression can be a simple choice for stacking)
meta_model = LogisticRegression()

# Stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model
)

# Evaluate Stacking performance
stacking_score = cross_val_score(stacking_clf, X_train, y_train, cv=10, scoring='accuracy').mean()

print(f"Stacking Classifier Accuracy: {stacking_score:.4f}")


### Comparison of All Methods

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# List all methods and their corresponding scores
methods = ['Bagging RF', 'Bagging ET', 'Bagging LGBM', 'Boosting RF', 'Boosting ET', 'Boosting LGBM', 'Stacking']
scores = [bagging_rf_score, bagging_et_score, bagging_lgbm_score, boosting_rf_score, boosting_et_score, boosting_lgbm_score, stacking_score]

# Plot bar chart
plt.figure(figsize=(10, 6))
sns.barplot(x=methods, y=scores, palette='viridis')

# Add percentage value labels on top of the bars
for i, v in enumerate(scores):
    plt.text(i, v + 0.002, f"{v * 100:.2f}%", ha='center', fontsize=12)

plt.ylim(min(scores) - 0.02, max(scores) + 0.02)
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison of Ensemble Methods')
plt.xticks(rotation=45)
plt.show()


## Save the Final Model

In [ ]:
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Best parameters
best_params = {
    'n_estimators': 1180,
    'max_depth': 17,
    'min_samples_split': 9,
    'min_samples_leaf': 1,
    'bootstrap': False,
    'random_state': 42
}

# Define Random Forest model with the best parameters
best_rf_clf = RandomForestClassifier(**best_params)

# Perform 10-fold cross-validation
cross_val_scores = cross_val_score(best_rf_clf, X_train, y_train, cv=10, scoring='accuracy')

# Print the cross-validation scores
print(f"Cross-validation scores: {cross_val_scores}")
print(f"Mean cross-validation score: {cross_val_scores.mean():.4f}")

# Fit the model on the entire training set
best_rf_clf.fit(X_train, y_train)

# Save the model
joblib.dump(best_rf_clf, 'best_rf_model.pkl')

print("Model saved as 'best_rf_model.pkl'")


### Deploy the Model using Flask

In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

# Initialize Flask app
app = Flask(__name__)

# Load the pre-trained model
model = joblib.load('best_rf_model.pkl')

# Define a prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    # Get the features from the request
    data = request.get_json()

    # Extract features from the data
    features = np.array(data['features']).reshape(1, -1)

    # Make a prediction using the model
    prediction = model.predict(features)

    # Return the prediction as a JSON response
    return jsonify({'prediction': int(prediction[0])})

if __name__ == '__main__':
    app.run(debug=True)


#                                                        THE END